In [71]:
import torch

In [2]:
# 텐서 생성
print(torch.tensor([[1,2],[3,4]]))

tensor([[1, 2],
        [3, 4]])


In [61]:
# GPU에 텐서 올리기
print(torch.tensor([[1,2],[3,4]],device = 'cuda'))
print(torch.tensor([[1,2],[3,4]],device = 'cuda').type()) #정수니까 longtensor

tensor([[1, 2],
        [3, 4]], device='cuda:0')
torch.cuda.LongTensor


In [60]:
print(torch.tensor([[1,2],[3,4]], device = 'cuda', dtype = torch.float64))
print(torch.tensor([[1,2],[3,4]], device = 'cuda', dtype = torch.float64).type()) #64로 지정했으니 doubletensor

tensor([[1., 2.],
        [3., 4.]], device='cuda:0', dtype=torch.float64)
torch.cuda.DoubleTensor


In [9]:
temp = torch.tensor([[1,2],[3,4]],device = 'cuda') 
print(temp)

tensor([[1, 2],
        [3, 4]], device='cuda:0')


In [11]:
# Numpy로 다시 변환
temp.to('cpu').numpy()

array([[1, 2],
       [3, 4]])

In [12]:
temp.to('cpu')

tensor([[1, 2],
        [3, 4]])

In [14]:
# 32비트 부동소수점 텐서 생성
temp = torch.FloatTensor([1,2,3,4,5,6,7])

In [15]:
print(temp[0])

tensor(1.)


In [16]:
print(temp[-1])

tensor(7.)


In [21]:
print(temp[3:-1])

tensor([4., 5., 6.])


In [25]:
temp = torch.tensor([[1,2],[3,4]])
print(temp.shape)
# reshape 2x2 to 4x1
print(temp.view(4,1))
# reshape to 1d tensor
print(temp.view(-1))

torch.Size([2, 2])
tensor([[1],
        [2],
        [3],
        [4]])
tensor([1, 2, 3, 4])


In [26]:
# reshape to 1 X ? tensor
# 니가 알아서 맞춰라는 뜻
# np의 reshape과 동일
print(temp.view(1,-1))

tensor([[1, 2, 3, 4]])


### read from csv

In [72]:
import os
os.getcwd()

'/content'

In [73]:
import pandas as pd
import numpy as np
house = pd.read_csv('sample_data/california_housing_train.csv')

In [74]:
house.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [75]:
house.shape

(17000, 9)

In [76]:
house.iloc[:,8]

0         66900.0
1         80100.0
2         85700.0
3         73400.0
4         65500.0
           ...   
16995    111400.0
16996     79000.0
16997    103600.0
16998     85800.0
16999     94600.0
Name: median_house_value, Length: 17000, dtype: float64

In [77]:
X = house[house.columns.difference(['median_house_value'])]
y = house['median_house_value']

In [78]:
X_torch = torch.from_numpy(np.array(X)).unsqueeze(dim = 1).float()

In [79]:
print(X_torch.type()) #float으로 지정해줬으니 float 텐서

torch.FloatTensor


# Build CustomDataset
: 배치화를 위해 사용하는 Torch의 DataLoader에 대한 연습

In [80]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [81]:
# CustomDataset 클래스 만들기
# CustomDataset 클래스 필요한 함수는 크게 3개다. init, len, getitem
# 
class CustomDataset(Dataset): # torch의 dataset을 상속!
  def __init__(self, csv_file): # init 함수 선언. 해당 클래스를 선헌하면, 바로 파일을 불러올 수 있게 설정
    self.label = pd.read_csv(csv_file)

  def __len__(self): #전체 데이터셋의 크기 반환하는 len 함수 선언
    return len(self.label) # 이미 init을 통해 class 선언시 label을 불러왔으니까, 여기서 바로 len도 계산!

  def __getitem(self, idx): #전체 데이터중에 해당 idx번의 데이터 가져오기
    sample = torch.tensor(self.label.iloc[idx, :8]).int()
    label = torch.tensor(self.label.iloc[idx, 8]).int() #longtypetensor로 구현하는구만
    return sample, label
# init

In [82]:
tensor_dataset = CustomDataset('sample_data/california_housing_train.csv')

In [83]:
dataset = DataLoader(tensor_dataset, batch_size = 4, shuffle = True)

In [84]:
dataset

# Define Model

In [85]:
import torch.nn as nn

In [87]:
# 함수를 통해 MLP 신경망을 생성해보자
# 책에 순서가 지맘대로 적혀있어서 앞으로 뺐다.

def MLP(in_features = 1, hidden_features = 20, out_features =1):
  hidden = nn.Linear(in_features = in_features, out_features = hidden_features, bias = True) #torch에서 선형회귀를 생성하는 모듈
  activation = nn.ReLU()
  output = nn.Linear(in_features = hidden_features, out_features = out_features, bias = True)
  net = nn.Sequential(hidden,
                      activation,
                      output)
  return net

In [90]:
# nn.Module을 상속해 딥러닝 학습에 필요한 __init__과 foward()를 포함하는 class 생성
class MLP(nn.Module):
  def __init__(self, inputs):
    super(MLP, self).__init__() #이거 정체가 뭐임...? 스스로를 상속하는 거임?
    self.layer = nn.Linear(inputs, 1)
    self.activation = nn.Sigmoid()

  def foward(self, X):
    X = self.layer(X)
    X = self.activation(X)
    return(X)

In [92]:
MLP(Dataset)
# 사실상 여기서 이 코드들은 그냥 더미라는 것이 밝혀짐;

TypeError: ignored

In [104]:
# nn.Sequential을 사용해 신경망 정의하기
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__() #부모 클래스의 메서드를 상속한다!
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 5),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(2)
    ) # 3 to 64의 Convolution Layer 생성

    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 30, kernel_size = 5),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(2)
    )

    self.layer3 = nn.Sequential(
        nn.Linear(in_features = 30*5*5, out_features = 10, bias = True),
        nn.ReLU(inplace = True)
    )
    
    def foward(self, x):
      x = self.layer1(x)
      x = self.layer2(x)
      x = x.view(x.shape[0],-1) # Linear에 넣기 위해 1d로 reshape
      x = self.layer3(x)

      return(x)

In [105]:
model = MLP()

In [106]:
list(model.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ), Sequential(
   (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
   (1): ReLU(inplace=True)
   (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ), Sequential(
   (0): Linear(in_features=750, out_features=10, bias=True)
   (1): ReLU(inplace=True)
 )]

# 모델 훈련
모델 훈련은 아래와 같은 순서로 이뤄진다
1. 모델, 손실함수, 옵티마이저 정의
2. 전방향학습 기울기 초기화
3. 전방향 학습 (출력 계산)
4. 오차 계산
5. 역전파 학습
6. 기울기 업데이트

In [111]:
# 모델 훈련 예시 코드
epochs= 100
crit = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer = optimizer, lr_lambda = lambda epoch:0.95 ** epoch)

for epoch in range(epochs):
  yhat = model(x_train) #출력 계산
  loss = crit(yhat, y_train) #오차 계싼
  optimizer.zero_grad() # 기울기 초기화 (오차의 중첩을 막기위해. torch는 기울기 누적이 기본)
  loss.backward() # 역전파
  optimizer.step() #기울기 업데이트!
  scheduler.step() #스케줄러 적용!
  

In [ ]:
# ?